In [1]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup as bs
import time
import pandas as pd

## 호텔 예약 주소 크롤링

In [ ]:
# 서울 호텔 검색 페이지 - 지역별
seoul_hotel = [
    'https://www.goodchoice.kr/product/search/2/2012',
    'https://www.goodchoice.kr/product/search/2/2019',
    'https://www.goodchoice.kr/product/search/2/2016',
    'https://www.goodchoice.kr/product/search/2/2014',
    'https://www.goodchoice.kr/product/search/2/2015',
    'https://www.goodchoice.kr/product/search/2/2020',
    'https://www.goodchoice.kr/product/search/2/2018',
    'https://www.goodchoice.kr/product/search/2/2017',
    'https://www.goodchoice.kr/product/search/2/2021'
]

In [ ]:
df = []

for link in link_list:
    a = link.split('&')[0]
    df.append(a)

linklist = pd.DataFrame({
    'hotel_link' : df
})

linklist.head

In [ ]:
linklist.to_csv('hotel_linklist.csv',index=False)

In [2]:
hotel_link = pd.read_csv('hotel_linklist.csv',index_col=False)
hotel_link.head()

,hotel_link
0,https://www.goodchoice.kr/product/detail?ano=6490
1,https://www.goodchoice.kr/product/detail?ano=1...
2,https://www.goodchoice.kr/product/detail?ano=6...
3,https://www.goodchoice.kr/product/detail?ano=6258
4,https://www.goodchoice.kr/product/detail?ano=4...


In [3]:
len(hotel_link)

294

## 크롤링 함수

In [4]:
# 크롤링 결과 담을 리스트

# 크롤링 함수
def crawl(start, end):
    for p in range(start,end):
        html = driver.page_source
        soup = bs(html, 'html.parser')
        
        reviews = soup.select('#review > ul > li')
        hotel_name = driver.find_element_by_css_selector("#content > div.top > div.right > div.info > h2").text
        
        for r in reviews:
            review.append(r.select_one('.txt').text)
            star.append(r.select_one('.score_wrap_sm .num').text)
            name.append(hotel_name)
        
        try : 
            driver.find_element_by_xpath('//*[@id="pagination"]/div/button['+str(p)+']').click()
        except : 
            pass
        time.sleep(2)
        #print(soup.select('.paging .on')[0].text)

def make_review_crawl(url):

    driver.get(url)

    # 페이지 파싱
    html = driver.page_source
    soup = bs(html, 'html.parser')

    # '리뷰' 클릭하도록 명령
    driver.find_element_by_xpath('//*[@id="content"]/div[2]/button[3]').click()
    
    top_text = soup.select('.score_top')[0].find('strong').text
    top_score = soup.select('.score_top .num')[0].text
    top_total=soup.select('.score_top')[0].find('p').find('b').text

    # 리뷰 총 개수
    top_total = int(soup.select('.score_top')[0].find('p').find('b').text)

    # 마지막 페이지 수 (한페이지에 리뷰 10개씩)
    if top_total % 10 == 0 :
        last_page = top_total // 10
    else : 
        last_page = top_total // 10  + 1

    # next_page 몇 번 클릭 해야하는지 설정
    if last_page % 5 == 0:
        page_step = last_page // 5 
    else:
        page_step = last_page // 5 + 1   
    
    print(top_total, last_page, page_step)
    step = 1
    while(page_step >= step):
        
        # 첫번째 step
        if step == 1:
            print('step1 loop')
            crawl(2,7)
            
        # 마지막 step
        elif step == page_step:
            print('step 마지막 loop')

                
            if top_total % 50 > 10:
                            
                if last_page % 5 == 0:
                    end_page = 7
                else: 
                    end_page =last_page % 5 + 2
                
                crawl(3, end_page)

            # 마지막 페이지
            html = driver.page_source
            soup = bs(html, 'html.parser')
            reviews = soup.select('#review > ul > li')
            hotel_name = driver.find_element_by_css_selector("#content > div.top > div.right > div.info > h2").text
        
            for r in reviews:
                review.append(r.select_one('.txt').text)
                star.append(r.select_one('.score_wrap_sm .num').text)
                name.append(hotel_name)    
        # 중간 step
        else:
            #print('step 중간 loop')
            crawl(3,8)
        step += 1

### 리뷰 크롤링  
- 예외 발생 시 링크를 retry_link 에 저장  
- 10개가 끝날 때 마다 df 데이터 프레임 만들어서 csv에 저장

In [6]:
review = []
name = []
star = []
retry_link = []

driver = webdriver.Chrome('./chromedriver')

#0에서 100 까지 다시 해야함 ㅠㅠ

for i in range(241,len(hotel_link)+1):
    a = hotel_link.iloc[i]['hotel_link']
    driver.implicitly_wait(3)
    try : 
        make_review_crawl(a)
        print(f'241부터 {i}번째 호텔 끝')
    except : 
        retry_link.append(a)
        print(f'{i}번째 호텔 못 끝냄')
    if i % 10 == 0:
        df = pd.DataFrame({
            '호텔명' : name,
            '리뷰' : review
        })
        df.to_csv(f'hotel_review_241부터_{i}th.csv',index=False, encoding='utf-8-sig')
    
    
driver.close()

1917 192 39
step1 loop


In [5]:
review = []
name = []
star = []
retry_link = []

driver = webdriver.Chrome('./chromedriver')

#0에서 100 까지 다시 해ㅠㅠ

for i in range(0,101):
    a = hotel_link.iloc[i]['hotel_link']
    driver.implicitly_wait(3)
    try : 
        make_review_crawl(a)
        print(f' {i}번째 호텔 끝')
    except : 
        retry_link.append(a)
        print(f'{i}번째 호텔 못 끝냄')
    if i % 10 == 0:
        df = pd.DataFrame({
            '호텔명' : name,
            '리뷰' : review
        })
        df.to_csv(f'hotel_review_0부터_{i}th.csv',index=False, encoding='utf-8-sig')
    
    
driver.close()

115 12 3
step1 loop
step 마지막 loop
 0번째 호텔 끝
649 65 13
step1 loop
step 마지막 loop
 1번째 호텔 끝
40 4 1
step1 loop
 2번째 호텔 끝
273 28 6
step1 loop
step 마지막 loop
 3번째 호텔 끝
469 47 10
step1 loop
step 마지막 loop
 4번째 호텔 끝
586 59 12
step1 loop
step 마지막 loop
 5번째 호텔 끝
1890 189 38
step1 loop
step 마지막 loop
 6번째 호텔 끝
432 44 9
step1 loop
step 마지막 loop
 7번째 호텔 끝
543 55 11
step1 loop
step 마지막 loop
 8번째 호텔 끝
807 81 17
step1 loop
step 마지막 loop
 9번째 호텔 끝
612 62 13
step1 loop
step 마지막 loop
 10번째 호텔 끝
86 9 2
step1 loop
step 마지막 loop
 11번째 호텔 끝
1040 104 21
step1 loop
step 마지막 loop
 12번째 호텔 끝
660 66 14
step1 loop
step 마지막 loop
 13번째 호텔 끝
650 65 13
step1 loop
step 마지막 loop
 14번째 호텔 끝
425 43 9
step1 loop
step 마지막 loop
 15번째 호텔 끝
1007 101 21
step1 loop
step 마지막 loop
 16번째 호텔 끝
102 11 3
step1 loop
step 마지막 loop
 17번째 호텔 끝
236 24 5
step1 loop
step 마지막 loop
 18번째 호텔 끝
92 10 2
step1 loop
step 마지막 loop
 19번째 호텔 끝
296 30 6
step1 loop
step 마지막 loop
 20번째 호텔 끝
1255 126 26
step1 loop
step 마지막 loop
 21번째 호텔 끝
122 13 3
step1 loop